In [10]:
import  os
print(os.getcwd())

/kaggle/working


# 在Kaggle上跑的准确率为94.5400%，由于结果过长所以无法在本地显示全部记录，在kaggle上已截图

In [ ]:
import torch
from torch import nn, optim
import torchvision
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
from torch.utils.data.sampler import SubsetRandomSampler
%matplotlib inline
import time
import numpy as np



show = ToPILImage() #可以把Tensor转换成Image,方便可视化
#训练集和验证集用随机剪裁
transform_train = transforms.Compose([  #transforms.Compose就是将对图像处理的方法集中起来
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),#水平翻转
    #transforms.RandomCrop((32, 32), padding=4),#对图片进行随机裁剪，裁剪的大小是32*32的，填充是4
#     参数：size- (sequence or int)，若为sequence,则为(h,w)，若为int，则(size,size)
#     padding-(sequence or int, optional)，此参数是设置填充多少个pixel。
#     当为int时，图像上下左右均填充int个，例如padding=4，则上下左右均填充4个pixel，若为32x32，则会变成40x40。
#     当为sequence时，若有2个数，则第一个数表示左右扩充多少，第二个数表示上下的。当有4个数时，则为左，上，右，下。
#     fill- (int or tuple) 填充的值是什么（仅当填充模式为constant时有用）。int时，各通道均填充该值，当长度为3的tuple时，表示RGB通道需要填充的值。
    transforms.ToTensor(),#转为Tensor
    #在做数据归一化之前必须要把PIL Image转成Tensor，而其他resize或crop操作则不需要。
     transforms.Normalize((0.5, 0.5 ,0.5), (0.5, 0.5, 0.5)),#归一化
#     归一化操作，这里有两个参数一个是均值，一个是方差，由于是RGB型的所以一个参有三个值，这里面的参数的大小是可调的，调节的公式是：
#     class torchvision.transforms.Normalize(mean, std)，
#     Normalized_image=(image-mean)/std
#     channel=（channel-mean）/std(因为transforms.ToTensor()已经把数据处理成[0,1],那么(x-0.5)/0.5
#     就是[-1.0, 1.0])这样一来，我们的数据中的每个值就变成了[-1,1]的数了。
    ])
#测试集要用中心剪裁
transform_test = transforms.Compose([  #transforms.Compose就是将对图像处理的方法集中起来
    transforms.Resize((224,224)),
    #transforms.RandomHorizontalFlip(),#水平翻转
    #transforms.CenterCrop((32, 32)),#对图片进行中心裁剪，裁剪的大小是32*32的
    transforms.ToTensor(),#转为Tensor
    #在做数据归一化之前必须要把PIL Image转成Tensor，而其他resize或crop操作则不需要。
     transforms.Normalize((0.5, 0.5 ,0.5), (0.5, 0.5, 0.5)),#归一化
    ])



# CIFAR-10数据集下载
train_data = datasets.CIFAR10(
    root='/kaggle/working',
                         train=True,                         # 训练集
                         # 数据变换(0, 255) -> (0, 1)
                         transform=transform_train
                              #download=True
                           )

test_data = datasets.CIFAR10(
    root='/kaggle/working',
                        train=False,                         # 测试集
                        transform=transform_test
                        #download=True
                          )

#划分训练集用于生成验证集
valid_size = 0.2
BATCH_SIZE = 160
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

#定义用于获取训练和验证批次的采样器
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# 使用DataLoader进行分批
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, sampler=train_sampler, num_workers=0)
valid_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, sampler=valid_sampler, num_workers=0)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE)

#数据集10个类的定义
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')	# 这里会存储迄今最优模型的参数
        self.val_loss_min = val_loss

# resnet34 Model
model = torchvision.models.resnet34(pretrained=True)

batch_size = 160
n_epochs = 500

# early stopping patience; how long to wait after last time validation loss improved.
patience = 10
    
#损失函数:这里用交叉熵
criterion = nn.CrossEntropyLoss()
#优化器 Adam
optimizer = optim.Adam(model.parameters(), lr=1e-4)

#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# to track the training loss as the model trains
train_losses = []
# to track the validation loss as the model trains
valid_losses = []
# to track the average training loss per epoch as the model trains
avg_train_losses = []
# to track the average validation loss per epoch as the model trains
avg_valid_losses = [] 

# initialize the early_stopping object
early_stopping = EarlyStopping(patience=patience, verbose=True)
 
for epoch in range(1, n_epochs + 1):

    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    start = time.time()
    # for batch, (data, target) in enumerate(train_loader, 1):
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # 前向传播
        #print('前向传播')
        # forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(inputs)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # calculate the loss
        loss = criterion(outputs, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # record training loss
        train_losses.append(loss.item())

    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    # for data, target in valid_loader:
    for i, data in enumerate(valid_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(inputs)
        # calculate the loss
        loss = criterion(output, labels)
        # record validation loss
        valid_losses.append(loss.item())

    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)
    
    epoch_len = len(str(n_epochs))
    
    print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                  f'train_loss: {train_loss:.5f} ' +
                  f'valid_loss: {valid_loss:.5f}')
    
    print(print_msg)
    print('using time is :{}'.format(time.time()-start))
    # clear lists to track next epoch
    train_losses = []
    valid_losses = []
    
    # early_stopping needs the validation loss to check if it has decresed, 
    # and if it has, it will make a checkpoint of the current model
    early_stopping(valid_loss, model)
    
    if early_stopping.early_stop:
        print("Early stopping")
        break
    
# load the last checkpoint with the best model
model.load_state_dict(torch.load('checkpoint.pt'))



# model, train_loss, valid_loss = train_model(model, batch_size, patience, n_epochs) 

#保存训练模型
torch.save(model, '/kaggle/working/cifar_10_resnet34.pt')
model = torch.load('/kaggle/working/cifar_10_resnet34.pt')

# 测试
model.eval()
correct = 0
total = 0

for data in test_loader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    # 前向传播
    out = model(images)
    _, predicted = torch.max(out.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()



#输出识别准确率
print('10000测试图像 准确率:{:.4f}%'.format(100 * correct / total)) 

[  1/500] train_loss: 0.61419 valid_loss: 0.18850
using time is :220.15036582946777
Validation loss decreased (inf --> 0.188499).  Saving model ...
[  2/500] train_loss: 0.12535 valid_loss: 0.16673
using time is :219.16784572601318
Validation loss decreased (0.188499 --> 0.166726).  Saving model ...
[  3/500] train_loss: 0.06866 valid_loss: 0.16084
using time is :217.00337195396423
Validation loss decreased (0.166726 --> 0.160845).  Saving model ...
[  4/500] train_loss: 0.05097 valid_loss: 0.15770
using time is :217.22592043876648
Validation loss decreased (0.160845 --> 0.157701).  Saving model ...
[  5/500] train_loss: 0.04141 valid_loss: 0.16633
using time is :219.93618965148926
EarlyStopping counter: 1 out of 10
[  6/500] train_loss: 0.03602 valid_loss: 0.16447
using time is :219.5357780456543
EarlyStopping counter: 2 out of 10
[  7/500] train_loss: 0.02741 valid_loss: 0.17197
using time is :220.04452300071716
EarlyStopping counter: 3 out of 10
[  8/500] train_loss: 0.02894 valid_l